In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import imageio
## Import everything needed to edit/save/watch video clips
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip, ImageClip
from IPython.display import HTML
%matplotlib inline

In [5]:
import math

def grayscale(img):
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
      
    cv2.fillPoly(mask, vertices, ignore_mask_color)
 
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):

    separate_lines(lines,img)
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):

    return cv2.addWeighted(initial_img, α, img, β, γ)

def Right_lane(img,right_fit,x_right, y_right):
    m_right = right_fit[0]
    b_right = right_fit[1]

    minright_x = min(x_right)
    maxright_x = max(x_right)

    minright_y = m_right*minright_x + b_right
    maxright_y = m_right*maxright_x + b_right

    Lane_R = cv2.line(img, (int(minright_x),int(minright_y)), (int(maxright_x), int(maxright_y)), (255, 0, 0), 10)
    
def Left_lane(img,left_fit,x_left, y_left):
    m_left = left_fit[0]
    b_left = left_fit[1]

    minleft_x = min(x_left)
    maxleft_x = max(x_left)

    minleft_y = m_left*minleft_x + b_left
    maxleft_y = m_left*maxleft_x + b_left

    Lane_L=cv2.line(img, (int(minleft_x), int(minleft_y)), (int(maxleft_x),int(maxleft_y)), (255, 0, 0), 10)
    
def separate_lines(lines,img):
   
    global x_leftbfr
    global y_leftbfr
    global x_rightbfr
    global y_rightbfr
    
    x_left=[]
    y_left=[]
    x_right=[]
    y_right=[]
    for line in lines:
        for x1,y1,x2,y2 in line:
            if(x1!=0 and x2!=0):
                slope = ((y2-y1)/(x2-x1))                
                if slope < 0.2  and x2 < 500 and x1<480:
                    x_left.append(x1)
                    x_left.append(x2)
                    y_left.append(y1)
                    y_left.append(y2)
                else :
                    x_right.append(x1)
                    x_right.append(x2)
                    y_right.append(y1)
                    y_right.append(y2)

    if(x_left and y_left):
        x_leftbfr=x_left
        y_leftbfr=y_left
        left_fit = np.polyfit(x_left, y_left,1)
        Left_lane(img,left_fit,x_left, y_left)
    else:
        left_fit = np.polyfit(x_leftbfr, y_leftbfr,1)
        Left_lane(img,left_fit,x_leftbfr, y_leftbfr)
        
    if(x_right and y_right):
        x_rightbfr = x_right
        y_rightbfr=y_right     
        right_fit = np.polyfit(x_right, y_right,1)
        Right_lane(img,right_fit,x_right, y_right)
    else:
        right_fit = np.polyfit(x_rightbfr, y_rightbfr,1)
        Right_lane(img,right_fit,x_rightbfr, y_rightbfr)

In [6]:
def pipeline(img):
        gray=grayscale(img)
        blur=gaussian_blur(gray, 7)
        edge=canny(blur,50, 150)
        imshape = img.shape
        mask=region_of_interest(edge,np.array([[(0,imshape[0]),(410,360), (590,360),(imshape[1],imshape[0])]], dtype=np.int32))
        
        lines = hough_lines(mask, 0.8, np.pi/180, 25, 50, 200)
        
        #print(lines)
        
        line_image = np.copy(img)
        Lane=weighted_img(lines,line_image)
        return Lane

In [7]:
####code for test videos
if os.path.isdir("test_videos_output"):
    pass
else:
    os.mkdir("test_videos_output")

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    result = pipeline(image)
    return result

white_output = 'C:\\Users\\Vivek.Pathak01\\Desktop\\Lane Lines\\CarND-LaneLines-P1-master\\test_videos_output\\solidYellowLeft.mp4'
clip1 = VideoFileClip("C:\\Users\\Vivek.Pathak01\\Desktop\\Lane Lines\\CarND-LaneLines-P1-master\\test_videos\\solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
clip1.reader.close()

items=os.listdir("C:\\Users\\Vivek.Pathak01\\Desktop\\Lane Lines\\CarND-LaneLines-P1-master\\test_images\\")
if os.path.isdir("test_images_output"):
    pass
else:
    os.mkdir("test_images_output")

for img in items:
    image = mpimg.imread('C:\\Users\\Vivek.Pathak01\\Desktop\\Lane Lines\\CarND-LaneLines-P1-master\\test_images\\'+img)
    image_output = process_image(image)
    mpimg.imsave(("C:\\Users\\Vivek.Pathak01\\Desktop\\Lane Lines\\CarND-LaneLines-P1-master\\test_images_output\\"+img),image_output,cmap='gray')

[MoviePy] >>>> Building video C:\Users\Vivek.Pathak01\Desktop\Lane Lines\CarND-LaneLines-P1-master\test_videos_output\solidYellowLeft.mp4
[MoviePy] Writing video C:\Users\Vivek.Pathak01\Desktop\Lane Lines\CarND-LaneLines-P1-master\test_videos_output\solidYellowLeft.mp4


100%|███████████████████████████████████████▉| 681/682 [01:37<00:00,  6.98it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:\Users\Vivek.Pathak01\Desktop\Lane Lines\CarND-LaneLines-P1-master\test_videos_output\solidYellowLeft.mp4 

Wall time: 1min 46s
